In [1]:
import pandas as pd
# import seaborn as sns
import numpy as np
import scipy.sparse as ss
# import ruamel.yaml as yaml
import os
import yaml, inspect, os, sys, subprocess, pprint, shutil, argparse



In [2]:
eyerissv2_pe_config_file = os.path.join(
    os.getcwd(),
    '../evaluation_setups/fig12_eyerissv2_pe_setup/hardware-specs/configurations/SI-SW.yaml'
)
eyerissv2_pe_art_file = os.path.join(
    os.getcwd(),
    '../evaluation_setups/fig12_eyerissv2_pe_setup/ert_art/ART.yaml'
)
eyerissv2_pe_ert_file = os.path.join(
    os.getcwd(),
    '../evaluation_setups/fig12_eyerissv2_pe_setup/ert_art/ERT.yaml'
)
eyerissv2_pe_arch_file = os.path.join(
    os.getcwd(),
    '../evaluation_setups/fig12_eyerissv2_pe_setup/hardware-specs/designs/arch.yaml'
)
# alexnet_conv_dir = os.path.join(
#     os.getcwd(),
#     '../evaluation_setups/table7_eyeriss_setup/workload/'
# ) 
# alexnet_conv_mapping_dir = os.path.join(
#     os.getcwd(),
#     '../evaluation_setups/table7_eyeriss_setup/mappings_found/'
# ) 
# spmspm_workloads_dir = os.path.join(
#     os.getcwd(),
#     '../evaluation_setups/fig1_format_comparison_setup/workload'
# ) 
# spmspm_workloads_mapping_dir = os.path.join(
#     os.getcwd(),
#     '../evaluation_setups/fig1_format_comparison_setup/workload'
# ) 
mobilenet_sparse_dir = os.path.join(
    os.getcwd(),
    '../evaluation_setups/fig12_eyerissv2_pe_setup/workload/mobilenet0.5-sparse/uniform'
)
mobilenet_sparse_mappings_dir = os.path.join(
    os.getcwd(),
    '../evaluation_setups/fig12_eyerissv2_pe_setup/mappings/mobilenet0.5-sparse/'
)

print(eyerissv2_pe_config_file,
eyerissv2_pe_art_file,
eyerissv2_pe_ert_file,
eyerissv2_pe_arch_file)

workload_file_array = []
mapping_file_array = []
# workload_dirs = [alexnet_conv_dir,spmspm_workloads_dir,mobilenet_sparse_dir]
workload_dirs = [mobilenet_sparse_dir]
mappings_dirs = [mobilenet_sparse_mappings_dir]



for workload_dir in workload_dirs:
    workload_file_array.extend([
        os.path.join(
            workload_dir,
            file 
        ) for file in os.listdir(workload_dir)
    ])
    
workload_file_array.sort()

for mapping_dir in mappings_dirs:
    mapping_file_array.extend([
        os.path.join(
            mapping_dir,
            file 
        ) for file in os.listdir(mapping_dir)
    ])

mapping_file_array.sort()

# print(workload_file_array)
# for workload in workload_file_array:
#     print(workload)

# for mapping in mapping_file_array:
#     print(mapping)

    
for workload,mapping in zip(workload_file_array,mapping_file_array):
    print(workload,mapping)
    print()

/home/workspace/2022.micro.artifact/notebook/../evaluation_setups/fig12_eyerissv2_pe_setup/hardware-specs/configurations/SI-SW.yaml /home/workspace/2022.micro.artifact/notebook/../evaluation_setups/fig12_eyerissv2_pe_setup/ert_art/ART.yaml /home/workspace/2022.micro.artifact/notebook/../evaluation_setups/fig12_eyerissv2_pe_setup/ert_art/ERT.yaml /home/workspace/2022.micro.artifact/notebook/../evaluation_setups/fig12_eyerissv2_pe_setup/hardware-specs/designs/arch.yaml
/home/workspace/2022.micro.artifact/notebook/../evaluation_setups/fig12_eyerissv2_pe_setup/workload/mobilenet0.5-sparse/uniform/L07.yaml /home/workspace/2022.micro.artifact/notebook/../evaluation_setups/fig12_eyerissv2_pe_setup/mappings/mobilenet0.5-sparse/L07-perfect_factor.yaml

/home/workspace/2022.micro.artifact/notebook/../evaluation_setups/fig12_eyerissv2_pe_setup/workload/mobilenet0.5-sparse/uniform/L09.yaml /home/workspace/2022.micro.artifact/notebook/../evaluation_setups/fig12_eyerissv2_pe_setup/mappings/mobilenet

In [3]:
import re

def parse_energy_summary(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    algorithmic = {
        'total_computes': None,
        'total_energy_per_compute_pJ': None,
        'breakdown': {}
    }
    actual = {
        'total_computes': None,
        'total_energy_per_compute_pJ': None,
        'breakdown': {}
    }

    in_algo = False
    in_actual = False

    for line in lines:
        stripped = line.strip()

        # Header flags
        if stripped.startswith("Algorithmic Computes ="):
            algorithmic['total_computes'] = int(stripped.split('=')[1].strip())
            in_algo = True
            in_actual = False
            continue

        if stripped.startswith("Actual Computes ="):
            actual['total_computes'] = int(stripped.split('=')[1].strip())
            in_algo = False
            in_actual = True
            continue

        # Breakdown parsing
        if in_algo and re.match(r'^.+?=\s+[\d.]+$', stripped):
            key, val = map(str.strip, stripped.split(' = '))
            val = float(val)
            if key == "Total":
                algorithmic['total_energy_per_compute_pJ'] = val
            else:
                algorithmic['breakdown'][key] = val

        elif in_actual and re.match(r'^.+?=\s+[\d.]+$', stripped):
            key, val = map(str.strip, stripped.split(' = '))
            val = float(val)
            if key == "Total":
                actual['total_energy_per_compute_pJ'] = val
            else:
                actual['breakdown'][key] = val

    return algorithmic, actual


In [4]:
def run_timeloop(workloads, mappings, config, pe_art, pe_ert, arch, output_dir):
    timeloop_exe = "timeloop-model"

    for workload,mapping in zip(workloads,mappings):
        
        
        
        lst_of_input_files = [
            workload,
            mapping,
            config,
            pe_art,
            pe_ert,
            arch
        ]
        # print(lst_of_input_files)
        logfile_path = os.path.join(output_dir, f"timeloop_'{workload.split('/')[-1]}'.log")
        
        subprocess_cmd = [timeloop_exe, *lst_of_input_files]
       
        outfile = open(logfile_path, "w+")    
        status = subprocess.call(subprocess_cmd) 
        outfile.close()
        
        mv_cmd = ['mv','timeloop-model.stats.txt',f'./debug/timeloop-model_"{workload.split("/")[-1]}".stats.txt']
        status = subprocess.call(mv_cmd) 
        
        algorithmic, actual = parse_energy_summary(f'./debug/timeloop-model_"{workload.split("/")[-1]}".stats.txt')
        print(algorithmic, actual)
        
        

In [5]:
output_dir = os.path.join(os.getcwd(),'./debug')
run_timeloop(
    workload_file_array,
    mapping_file_array,
    eyerissv2_pe_config_file,
    eyerissv2_pe_art_file,
    eyerissv2_pe_ert_file,
    eyerissv2_pe_arch_file,
    output_dir)

Utilization = 1.00 | pJ/Algorithmic-Compute =    1.190 | pJ/Compute =    3.135
{'total_computes': 4194304, 'total_energy_per_compute_pJ': 1.19, 'breakdown': {'MAC': 0.22, 'reg': 0.09, 'psum_spad': 0.3, 'weight_spad': 0.54, 'iact_spad': 0.05, 'BackingStorage': 0.0, 'BackingStorage <==> iact_spad': 0.0, 'iact_spad <==> weight_spad': 0.0, 'psum_spad <==> reg': 0.0, 'reg <==> MAC': 0.0, 'weight_spad <==> psum_spad': 0.0}} {'total_computes': 1592245, 'total_energy_per_compute_pJ': 3.14, 'breakdown': {'MAC': 0.58, 'reg': 0.23, 'psum_spad': 0.78, 'weight_spad': 1.41, 'iact_spad': 0.13, 'BackingStorage': 0.0, 'BackingStorage <==> iact_spad': 0.0, 'iact_spad <==> weight_spad': 0.0, 'psum_spad <==> reg': 0.0, 'reg <==> MAC': 0.0, 'weight_spad <==> psum_spad': 0.0}}
Utilization = 1.00 | pJ/Algorithmic-Compute =    1.792 | pJ/Compute =    2.540
{'total_computes': 2097152, 'total_energy_per_compute_pJ': 1.79, 'breakdown': {'MAC': 0.4, 'reg': 0.1, 'psum_spad': 0.5, 'weight_spad': 0.76, 'iact_spad': 

Utilization = 1.00 | pJ/Algorithmic-Compute =    1.429 | pJ/Compute =    2.689
{'total_computes': 2097152, 'total_energy_per_compute_pJ': 1.43, 'breakdown': {'MAC': 0.3, 'reg': 0.1, 'psum_spad': 0.39, 'weight_spad': 0.61, 'iact_spad': 0.03, 'BackingStorage': 0.0, 'BackingStorage <==> iact_spad': 0.0, 'iact_spad <==> weight_spad': 0.0, 'psum_spad <==> reg': 0.0, 'reg <==> MAC': 0.0, 'weight_spad <==> psum_spad': 0.0}} {'total_computes': 1114139, 'total_energy_per_compute_pJ': 2.69, 'breakdown': {'MAC': 0.57, 'reg': 0.18, 'psum_spad': 0.74, 'weight_spad': 1.15, 'iact_spad': 0.05, 'BackingStorage': 0.0, 'BackingStorage <==> iact_spad': 0.0, 'iact_spad <==> weight_spad': 0.0, 'psum_spad <==> reg': 0.0, 'reg <==> MAC': 0.0, 'weight_spad <==> psum_spad': 0.0}}


Utilization = 1.00 | pJ/Algorithmic-Compute =    1.028 | pJ/Compute =    3.064
{'total_computes': 4194304, 'total_energy_per_compute_pJ': 1.03, 'breakdown': {'MAC': 0.19, 'reg': 0.07, 'psum_spad': 0.27, 'weight_spad': 0.47, 'iact_spad': 0.02, 'BackingStorage': 0.0, 'BackingStorage <==> iact_spad': 0.0, 'iact_spad <==> weight_spad': 0.0, 'psum_spad <==> reg': 0.0, 'reg <==> MAC': 0.0, 'weight_spad <==> psum_spad': 0.0}} {'total_computes': 1407304, 'total_energy_per_compute_pJ': 3.06, 'breakdown': {'MAC': 0.58, 'reg': 0.21, 'psum_spad': 0.8, 'weight_spad': 1.41, 'iact_spad': 0.06, 'BackingStorage': 0.0, 'BackingStorage <==> iact_spad': 0.0, 'iact_spad <==> weight_spad': 0.0, 'psum_spad <==> reg': 0.0, 'reg <==> MAC': 0.0, 'weight_spad <==> psum_spad': 0.0}}


Utilization = 1.00 | pJ/Algorithmic-Compute =    1.136 | pJ/Compute =    2.958
{'total_computes': 4194304, 'total_energy_per_compute_pJ': 1.14, 'breakdown': {'MAC': 0.22, 'reg': 0.07, 'psum_spad': 0.3, 'weight_spad': 0.52, 'iact_spad': 0.02, 'BackingStorage': 0.0, 'BackingStorage <==> iact_spad': 0.0, 'iact_spad <==> weight_spad': 0.0, 'psum_spad <==> reg': 0.0, 'reg <==> MAC': 0.0, 'weight_spad <==> psum_spad': 0.0}} {'total_computes': 1610668, 'total_energy_per_compute_pJ': 2.96, 'breakdown': {'MAC': 0.57, 'reg': 0.19, 'psum_spad': 0.78, 'weight_spad': 1.36, 'iact_spad': 0.06, 'BackingStorage': 0.0, 'BackingStorage <==> iact_spad': 0.0, 'iact_spad <==> weight_spad': 0.0, 'psum_spad <==> reg': 0.0, 'reg <==> MAC': 0.0, 'weight_spad <==> psum_spad': 0.0}}


Utilization = 1.00 | pJ/Algorithmic-Compute =    1.248 | pJ/Compute =    2.922
{'total_computes': 4194304, 'total_energy_per_compute_pJ': 1.25, 'breakdown': {'MAC': 0.24, 'reg': 0.07, 'psum_spad': 0.33, 'weight_spad': 0.59, 'iact_spad': 0.02, 'BackingStorage': 0.0, 'BackingStorage <==> iact_spad': 0.0, 'iact_spad <==> weight_spad': 0.0, 'psum_spad <==> reg': 0.0, 'reg <==> MAC': 0.0, 'weight_spad <==> psum_spad': 0.0}} {'total_computes': 1791135, 'total_energy_per_compute_pJ': 2.92, 'breakdown': {'MAC': 0.57, 'reg': 0.16, 'psum_spad': 0.77, 'weight_spad': 1.37, 'iact_spad': 0.05, 'BackingStorage': 0.0, 'BackingStorage <==> iact_spad': 0.0, 'iact_spad <==> weight_spad': 0.0, 'psum_spad <==> reg': 0.0, 'reg <==> MAC': 0.0, 'weight_spad <==> psum_spad': 0.0}}
Utilization = 1.00 | pJ/Algorithmic-Compute =    1.294 | pJ/Compute =    2.926
{'total_computes': 2097152, 'total_energy_per_compute_pJ': 1.29, 'breakdown': {'MAC': 0.25, 'reg': 0.08, 'psum_spad': 0.34, 'weight_spad': 0.6, 'iact_spad